In [3]:
import os
import shutil
import pandas as pd  # To read the Excel files and check the number of sheets
import xlrd  # To read the .xls files and check cell values
from datetime import datetime

source_dir = r'C:\Users\mvalsania25\Desktop\Parking_Complete'
# Create the destination directories for the sorted files
destination_dir_x = r'C:\Users\mvalsania25\Desktop\Destination_X'
destination_dir_y = r'C:\Users\mvalsania25\Desktop\Destination_Y'

# Ensure the destination directories exist
os.makedirs(destination_dir_x, exist_ok=True)
os.makedirs(destination_dir_y, exist_ok=True)

# List to keep track of folders with Excel files that didn't meet any condition
folders_without_copy = []

# Set the Excel file extensions
excel_extensions = ('.xls', '.xlsx')

# Function to check the number of sheets in an Excel file
def get_excel_sheet_count(file_path):
    try:
        df = pd.read_excel(file_path, sheet_name=None)  # Load all sheets in the Excel file
        return len(df)  # Return the number of sheets
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return 0  # Return 0 if an error occurs

# Function to check if cell F16 ends with ":00" in an Excel file
def check_f16_ends_with_00(file_path):
    try:
        # Open the Excel file using xlrd (only for .xls files)
        if file_path.lower().endswith('.xls'):
            workbook = xlrd.open_workbook(file_path)
            sheet = workbook.sheet_by_index(0)  # Assuming we're working with the first sheet
            
            # Get the value of cell F16 (row 15, column 5 in 0-based indexing)
            cell_value = sheet.cell_value(15, 5)
            
            # Check if the cell value is a string and if it ends with ":00"
            if isinstance(cell_value, str) and cell_value.endswith(':00'):
                return True
            elif isinstance(cell_value, (int, float)) and str(int(cell_value)).endswith(':00'):
                return True
        return False
    except Exception as e:
        print(f"Error reading F16 from {file_path}: {e}")
        return False

# Helper function to check if the word "transaction" appears twice in the file name
def contains_transaction_twice(file_name):
    return file_name.lower().count('trans') >= 2

# Function to handle file name conflicts
def get_unique_file_path(destination_folder, file_name):
    base_name, extension = os.path.splitext(file_name)
    counter = 1
    new_file_name = file_name
    
    # Check if the file already exists in the destination folder
    while os.path.exists(os.path.join(destination_folder, new_file_name)):
        new_file_name = f"{base_name}_{counter}{extension}"
        counter += 1
    
    return new_file_name

# Helper function to check for any variations of "criteria" in the file name
def contains_criteria(file_name):
    criteria_variations = ['criteria', 'citeria', 'criertia', 'criteeria', 'critiera', 'criteera', 'creiteira', 'critieira', 'criterita']
    return any(variation in file_name.lower() for variation in criteria_variations)

# Iterate through the subdirectories
for root, dirs, files in os.walk(source_dir):

    # Skip the directory if it contains "daily" or "payroll" in its name
    if any(keyword in os.path.basename(root).lower() for keyword in ['daily', 'payroll']):
        print(f"Skipping directory: {root}")
        continue  # Skip this directory and move to the next one

    # Check if there's at least one Excel file in the directory
    excel_files_in_dir = [file for file in files if file.lower().endswith(excel_extensions)]
    if excel_files_in_dir:
        # Store the full path of the immediate subdirectory
        subdirectory_full_path = os.path.abspath(root)
        
        # Initialize a flag to track if any condition is met
        conditions_met = 0
        
        # Iterate through all the Excel files in the directory
        for file in excel_files_in_dir:
            file_path = os.path.join(root, file)
            
            # Get the number of sheets in the Excel file
            sheet_count = get_excel_sheet_count(file_path)
            
            # Check if F16 ends with ":00"
            if check_f16_ends_with_00(file_path):
                # If it ends with ":00", we proceed to check other conditions
                if contains_criteria(file.lower()) and sheet_count == 2:
                    # Condition 1: Criteria in the name, and exactly 2 sheets
                    dest_file_path = os.path.join(destination_dir_x, get_unique_file_path(destination_dir_x, file))
                    shutil.copy2(file_path, dest_file_path)
                    print(f"Moved {file} to Destination_X")
                    conditions_met += 1
                    
                elif contains_criteria(file.lower()) and sheet_count > 2:
                    # Condition 2: Criteria in the name, and more than 2 sheets
                    dest_file_path = os.path.join(destination_dir_y, get_unique_file_path(destination_dir_y, file))
                    shutil.copy2(file_path, dest_file_path)
                    print(f"Moved {file} to Destination_Y")
                    conditions_met += 1
                    
                # First condition: Checking for "revenue" with more than 2 sheets
                elif 'revenue' in file.lower() and not any(keyword in file.lower() for keyword in ['all', 'lot', 'park', 'exception']) and sheet_count > 2 and not contains_transaction_twice(file.lower()):
                    # Condition 3: "Revenue" in the name, other conditions and more than 2 sheets, but not "transaction" twice
                    dest_file_path = os.path.join(destination_dir_y, get_unique_file_path(destination_dir_y, file))
                    shutil.copy2(file_path, dest_file_path)
                    print(f"Moved {file} to Destination_Y")
                    conditions_met += 1

                # Second condition: Checking for "revenue" with exactly 2 sheets
                elif 'revenue' in file.lower() and not any(keyword in file.lower() for keyword in ['all', 'lot', 'park', 'exception']) and sheet_count == 2 and not contains_transaction_twice(file.lower()):
                    # Condition 4: "Revenue" in the name, other conditions and exactly 2 sheets, but not "transaction" twice
                    dest_file_path = os.path.join(destination_dir_x, get_unique_file_path(destination_dir_x, file))
                    shutil.copy2(file_path, dest_file_path)
                    print(f"Moved {file} to Destination_X")
                    conditions_met += 1
        
        # After iterating through all files, check if no condition was met or if the folder was clearly one that was not meant to contain a meaningful file
        if conditions_met == 0 and 'revis' not in os.path.basename(root).lower() and os.path.basename(root) != os.path.basename(os.path.dirname(root)):
            folders_without_copy.append(subdirectory_full_path)

# At the end, print the list of folders that contained Excel files but did not meet any conditions
if folders_without_copy:
    print("\nFolders with Excel files that did not meet any conditions:")
    for folder in folders_without_copy:
        print(f"- {folder}")
else:
    print("\nAll directories with Excel files met at least one condition.")


Moved Revenue Transaction Detail_ CRITERIA 7-1-2022.xls to Destination_Y
Moved Revenue Transaction Detail_ CRITERIA 7-10-2022.xls to Destination_Y
Moved Revenue Transaction Detail_ 07-11-2022.xls to Destination_Y
Moved Revenue Transaction Detail_CRITERIA_.xls to Destination_X
Moved Revenue Transaction Detail_CRITERIA_.xls to Destination_X
Moved Revenue Transaction Detail_CRITERIA_.xls to Destination_X
Moved Revenue Transaction Detail_ CRITERIA 7-15-2022.xls to Destination_Y
Moved Revenue Transaction Detail_CRITERIA_.xls to Destination_X
Moved Revenue Transaction Detail_ CRITERIA 7-17-22.xls to Destination_X
Moved Revenue Transaction Detail_CRITERIA_.xls to Destination_X
Moved Revenue Transaction Detail_CRITERIA_.xls to Destination_X
Moved Revenue Transaction Detail_CRITERIA_.xls to Destination_X
Moved Revenue Transaction Detail_CRITERIA_.xls to Destination_X
Moved Revenue Transaction Detail_ CRITERIA 7-22-2022.xls to Destination_Y
Moved Revenue Transaction Detail_ CRITERIA 7-23-2022.xl